In [39]:
from llama_index import GPTVectorStoreIndex, download_loader, SimpleDirectoryReader, LLMPredictor, PromptHelper,ServiceContext
from langchain.agents import initialize_agent, Tool
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
import openai
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.embeddings import OpenAIEmbeddings
from langchain.agents import create_pandas_dataframe_agent
from llama_index import LangchainEmbedding
import os
import pandas as pd

设置openAI 相关的环境变量

In [40]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://openaidemo-hu.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "a65f52d60c744eb9b141d9939cd4c4b6"


openai.api_type = "azure"
openai.api_version = "2023-05-15"
openai.api_base = "https://openaidemo-hu.openai.azure.com/"
openai.api_key = "a65f52d60c744eb9b141d9939cd4c4b6"

davinci = AzureOpenAI(
    deployment_name="text-davinci-003",
    model_name="text-davinci-003",
    temperature=0, 
    top_p=1
)

gpt4 = AzureChatOpenAI(
    openai_api_base=openai.api_base,
    openai_api_version="2023-05-15",
    deployment_name="gpt4",
    openai_api_key=openai.api_key,
    openai_api_type=openai.api_type,
    temperature=0
)


embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1, openai_api_version="2023-05-15")

读1000条记录的sample文件，并且对title进行向量化

In [41]:
df = pd.read_csv("./azure_title_test_2.csv", encoding='utf-8')

df["title_embedding"] = embeddings.embed_documents(df["title"].tolist())

把内容保存到新csv文件中，加快多次测试数据，节省token数量

In [42]:
df.to_csv("azure_title_test_2_embedding.csv", index=False, encoding='utf-8')


把向量的余弦值计算出来

In [43]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain.embeddings import OpenAIEmbeddings

# Convert the list of vectors to a numpy array
embedded_vectors = np.array(df["title_embedding"].tolist())

# Calculate the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(embedded_vectors)


# 将二维数组转换为 DataFrame
df = pd.DataFrame(cosine_sim_matrix)

In [44]:
df.to_csv('azure_title_test_2_embedding_cosine_sim_matrix.csv', index=False)

csv文件里面记录，有一些标题是一模一样，这样的是不是要去重？ 然后定义一个事件是不是有多篇相似报道，才算是一个事件？

所以 0.999999>= cosine >= 0.9，数量超过3个才算一个事件？

Get the row and column indices of all elements in the dataframe that satisfy the condition

In [9]:
df = pd.read_csv("./title_embedding_cosine_sim_matrix.csv", encoding='utf-8')

In [45]:
import numpy as np

# 使用triu函数过滤掉下三角部分
mask = np.triu(np.ones(df.shape), k=1).astype(bool)

# 将DataFrame与掩码相乘以过滤掉下三角部分
resultDf = df.mask(mask)

filteredIndex=[]

for index, row in resultDf.iterrows():
    rowIndex = [index]
    for i in range(len(row)):
        if row[i] > 0.9 and row[i] < 0.99999:
            rowIndex.append(i)
    filteredIndex.append(rowIndex)

# # Combine the row and column indices into a list of tuples
# index_tuples = list(zip(indices[0], indices[1]))

# Print the list of tuples
#print(filteredIndex)

In [46]:
filteredIndexSetList = []
for element in filteredIndex:
    filteredIndexSetList.append(set(element))

过滤出大于等于15条新闻的内容，为一个事件

In [47]:
matchedIndex = []

duplicatedIndex = []
for index, element in enumerate(filteredIndexSetList):
    for s2 in filteredIndexSetList:
        if element != s2 and element.issubset(s2):
            duplicatedIndex.append(index)
            break

new_list = [value for index, value in enumerate(filteredIndex) if index not in duplicatedIndex]

for element in new_list :
    if len(element) >= 15 :
        matchedIndex.append(element)

print(len(matchedIndex))
print(matchedIndex)

356
[[157, 96, 100, 103, 104, 106, 107, 109, 112, 114, 115, 118, 120, 121, 122, 123, 127, 128, 130, 131, 133, 134, 137, 139, 141, 144, 145, 149, 150, 153, 154], [188, 149, 166, 167, 168, 169, 170, 171, 172, 173, 176, 177, 178, 179, 180, 182, 183, 184, 187], [769, 53, 361, 362, 363, 365, 366, 367, 369, 370, 371, 437, 438, 494, 545, 547, 650], [788, 53, 361, 362, 365, 371, 372, 373, 437, 494, 545, 547, 650, 769, 778, 784, 787], [823, 579, 580, 595, 622, 655, 695, 699, 702, 724, 800, 805, 806, 807, 809, 816], [825, 579, 580, 595, 622, 655, 695, 699, 702, 724, 800, 805, 806, 807, 809, 816], [1190, 361, 362, 363, 365, 366, 367, 369, 370, 371, 437, 438, 494, 545, 650, 769, 775, 784, 787, 788, 796, 832, 833], [1285, 53, 362, 363, 365, 366, 367, 371, 437, 438, 494, 545, 547, 650, 769, 772, 775, 778, 784, 787, 788, 832, 833, 1190, 1191, 1192], [1335, 365, 368, 371, 437, 494, 769, 771, 784, 792, 794, 796, 797, 817, 1025, 1190, 1192, 1198, 1199, 1201, 1285], [1336, 365, 366, 437, 494, 769, 784, 7

去掉重复元素超过80%的集合，总数个数较小的那个，如果总数相同，那就去掉首index较小的那个

In [48]:
matchedIndexSetList = []
for element in matchedIndex:
    matchedIndexSetList.append(set(element))

to_remove = set()

for i in range(len(matchedIndexSetList)):
    for j in range(i + 1, len(matchedIndexSetList)):
        common_elements = len(matchedIndexSetList[i].intersection(matchedIndexSetList[j]))
        if common_elements > 0.6 * len(matchedIndexSetList[i]):
            if len(matchedIndexSetList[i]) < len(matchedIndexSetList[j]):
                to_remove.add(i)
            else:
                to_remove.add(j)

overlayed_filtered_list = [matchedIndex[i] for i in range(len(matchedIndex)) if i not in to_remove]

print(len(overlayed_filtered_list))
print(overlayed_filtered_list)

82
[[157, 96, 100, 103, 104, 106, 107, 109, 112, 114, 115, 118, 120, 121, 122, 123, 127, 128, 130, 131, 133, 134, 137, 139, 141, 144, 145, 149, 150, 153, 154], [188, 149, 166, 167, 168, 169, 170, 171, 172, 173, 176, 177, 178, 179, 180, 182, 183, 184, 187], [823, 579, 580, 595, 622, 655, 695, 699, 702, 724, 800, 805, 806, 807, 809, 816], [1335, 365, 368, 371, 437, 494, 769, 771, 784, 792, 794, 796, 797, 817, 1025, 1190, 1192, 1198, 1199, 1201, 1285], [1338, 784, 796, 819, 832, 833, 1198, 1275, 1276, 1294, 1299, 1303, 1304, 1311, 1327, 1335, 1336, 1337], [1339, 53, 363, 365, 366, 367, 371, 437, 438, 494, 545, 650, 769, 775, 778, 784, 787, 788, 791, 802, 832, 833, 1190, 1191, 1192, 1198, 1285, 1335, 1336], [1597, 494, 548, 550, 551, 769, 792, 794, 796, 819, 829, 1199, 1201, 1336, 1337, 1338], [2158, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 

根据选定的内容，生成事件

In [49]:
df = pd.read_csv("./azure_title_test_2.csv", encoding='utf-8')

filteredEventSourceList = []

for indexRow in overlayed_filtered_list:
    filteredEventSource = pd.DataFrame(columns=df.columns)
    for  element in indexRow:
        filteredEventSource.loc[len(filteredEventSource)] = df.iloc[element]
    filteredEventSourceList.append(filteredEventSource)

print(filteredEventSourceList)

[     post_id                                 title        news_time  \
0   25648287   8月2日永鼎股份涨5.31%，富国融悦12个月持有期混合A基金重仓该股  1690960000000.0   
1   25648061      8月2日华大九天涨5.77%，长城久嘉创新成长混合A基金重仓该股  1690960000000.0   
2   25648068        8月2日迈为股份涨6.07%，国富潜力组合混合A基金重仓该股  1690960000000.0   
3   25648075           8月2日赛微电子跌8.91%，国都聚成混合基金重仓该股  1690960000000.0   
4   25648076      8月2日中科电气涨10.31%，长城创业板指数增强A基金重仓该股  1690960000000.0   
5   25648080   8月2日诺普信创60日新低，国泰君安中证1000指数增强A基金重仓该股  1690960000000.0   
6   25648096       8月2日巨星科技涨5.85%，交银经济新动力混合A基金重仓该股  1690960000000.0   
7   25648110      8月2日西部材料跌5.41%，大成睿景灵活配置混合A基金重仓该股  1690960000000.0   
8   25648120      8月2日艾为电子涨5.16%，汇丰晋信动态策略混合A基金重仓该股  1690960000000.0   
9   25648125     8月2日东威科技涨11.78%，汇丰晋信低碳先锋股票A基金重仓该股  1690960000000.0   
10  25648129     8月2日四方光电涨5.19%，天弘创新成长混合发起式A基金重仓该股  1690960000000.0   
11  25648149        8月2日茂莱光学涨10.10%，诺安优化配置混合基金重仓该股  1690960000000.0   
12  25648152         8月2日美芯晟涨7.17%，长城创新驱动混合A基金重仓该股  1690960000000.0   
13  2

In [50]:
eventSources = []
for events in filteredEventSourceList:
    eventSource=""
    for index, event in events.iterrows():
        rowNo = str(index +1) + ". "
        eventSource = eventSource + rowNo + event["title"] + "\n "
    eventSources.append(eventSource)

print(eventSources)

['1. 8月2日永鼎股份涨5.31%，富国融悦12个月持有期混合A基金重仓该股\n 2. 8月2日华大九天涨5.77%，长城久嘉创新成长混合A基金重仓该股\n 3. 8月2日迈为股份涨6.07%，国富潜力组合混合A基金重仓该股\n 4. 8月2日赛微电子跌8.91%，国都聚成混合基金重仓该股\n 5. 8月2日中科电气涨10.31%，长城创业板指数增强A基金重仓该股\n 6. 8月2日诺普信创60日新低，国泰君安中证1000指数增强A基金重仓该股\n 7. 8月2日巨星科技涨5.85%，交银经济新动力混合A基金重仓该股\n 8. 8月2日西部材料跌5.41%，大成睿景灵活配置混合A基金重仓该股\n 9. 8月2日艾为电子涨5.16%，汇丰晋信动态策略混合A基金重仓该股\n 10. 8月2日东威科技涨11.78%，汇丰晋信低碳先锋股票A基金重仓该股\n 11. 8月2日四方光电涨5.19%，天弘创新成长混合发起式A基金重仓该股\n 12. 8月2日茂莱光学涨10.10%，诺安优化配置混合基金重仓该股\n 13. 8月2日美芯晟涨7.17%，长城创新驱动混合A基金重仓该股\n 14. 8月2日智翔金泰-U跌7.44%，汇添富中证上海国企ETF基金重仓该股\n 15. 8月2日禾川科技涨6.41%，永赢先进制造智选混合发起A基金重仓该股\n 16. 8月2日财富趋势涨7.20%，南方金融主题灵活配置混合A基金重仓该股\n 17. 8月2日泽璟制药-U跌5.14%，金鹰医疗健康产业A基金重仓该股\n 18. 8月2日柏楚电子涨5.08%，易方达新收益混合A基金重仓该股\n 19. 8月2日唯捷创芯涨5.27%，诺安和鑫灵活配置混合基金重仓该股\n 20. 8月2日方邦股份涨9.06%，长城久恒灵活配置混合A基金重仓该股\n 21. 8月2日联德股份涨5.37%，诺安先锋混合A基金重仓该股\n 22. 8月2日新华网涨8.60%，天治核心成长混合(LOF)基金重仓该股\n 23. 8月2日康辰药业跌8.08%，华宝医药生物混合基金重仓该股\n 24. 8月2日司太立跌6.51%，银河医药混合A基金重仓该股\n 25. 8月2日杰克股份连续4日创60日新高，天弘永利债券A基金重仓该股\n 26. 8月2日维力医疗跌5.23%，富国医疗保健行业混合A基金重仓该股\n 27. 8

根据一组新闻标题，让大语言模型生成一个事件

In [52]:
from openAIRoundRobin.eventCreationFromTitle import  eventCreateByTitle

eventFinalList =[]
for eventSource in eventSources:
    eventFinalList.append(eventCreateByTitle(eventSource))

print(eventFinalList)


openai index: 0
8月2日金融头条：多家基金重仓涨跌股，关注医疗、科技、房地产等板块。
openai index: 1
8月2日热股涨停：金融科技、国产软件、房地产等概念领涨。
openai index: 2
BAE Systems reports strong 1H results, raises dividend, and launches £1.5B share buyback program.
openai index: 3
Mixed financial results for Japanese companies in 1H and FY earnings.
openai index: 4
Mitsubishi Chemical reports mixed results; Sumitomo faces losses; weak demand impacts revenues.
openai index: 5
Mixed 1Q results for Japanese companies; some profits rise, others decline or report losses.
openai index: 0
Various companies report mixed financial results and forecasts.
openai index: 1
8月1日多家公司融资净买入大幅增长，恒瑞医药、城建发展等排名前列。
openai index: 2
日本央行副行长：将调整债券购买，抑制收益率过快上升。
openai index: 3
恒生科技指数跌幅扩大，全球股市普遍下跌。
openai index: 4
南向资金8月1日：多家公司获净买入，部分股票连续增持。
openai index: 5
港股科技股跌幅扩大，恒指跌2.2%，银行股、电商股走弱。
openai index: 0
8月2日：全球股指期货下跌，A50跌1%，科技股领跌。
openai index: 1
8月1日南向资金加仓多家公司，中泰证券净卖出191.21万元。
openai index: 2
各大基金密集调研香飘飘、亿嘉和等热门个股。
openai index: 3
日本央行副行长内田真一：维持宽松政策，审慎调整收益率曲线控制。
openai

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


Q2 2023 Results: Major Companies Report Earnings, Show Optimistic Outlook
openai index: 1
Uber Q2: $9.2B revenue, 15% YoY growth, record bookings, and first-ever GAAP profit.
openai index: 2
Major companies report Q2 2023 earnings, mixed results seen.
openai index: 3
截至2023年7月31日，多家上市公司股东户数公布，关注市场变化。
['8月2日金融头条：多家基金重仓涨跌股，关注医疗、科技、房地产等板块。', '8月2日热股涨停：金融科技、国产软件、房地产等概念领涨。', 'BAE Systems reports strong 1H results, raises dividend, and launches £1.5B share buyback program.', 'Mixed financial results for Japanese companies in 1H and FY earnings.', 'Mitsubishi Chemical reports mixed results; Sumitomo faces losses; weak demand impacts revenues.', 'Mixed 1Q results for Japanese companies; some profits rise, others decline or report losses.', 'Various companies report mixed financial results and forecasts.', '8月1日多家公司融资净买入大幅增长，恒瑞医药、城建发展等排名前列。', '日本央行副行长：将调整债券购买，抑制收益率过快上升。', '恒生科技指数跌幅扩大，全球股市普遍下跌。', '南向资金8月1日：多家公司获净买入，部分股票连续增持。', '港股科技股跌幅扩大，恒指跌2.2%，银行股、电商股走弱。', '8月2日：全球股指期货下跌，A50跌1%，科技股领跌。', '8月1日南

把时间和原来的数据进行merge到一个dataframe中来，然后保存到csv文件

In [53]:
finalEventResult = pd.DataFrame(columns=["event",df.columns])

for  index,filteredEventSource in enumerate(filteredEventSourceList):
    col_new = pd.Series([eventFinalList[index]])        
    filteredEventSource.insert(0,'event',col_new)
    #pd.concat([finalEventResult,filteredEventSource],ignore_index=True, sort=False)
    #finalEventResult.append(filteredEventSource)

finalEventResult = pd.concat(filteredEventSourceList,ignore_index=True, sort=False)
#print(finalEventResult)


In [38]:
finalEventResult.to_csv("azure_title_test_2_event_create_by_title.csv", index=False, encoding="utf-8-sig")

In [5]:
from openAIRoundRobin import *

eventCreateByTitle("hello world.")

openai index: 1
Hello world headline lacks financial context.


'Hello world headline lacks financial context.'